<a href="https://colab.research.google.com/github/mancinimassimiliano/DeepLearningLab/blob/master/myFirstNN_solution_with_vis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

# Library needed for visualization purposes
from tensorboardcolab import TensorBoardColab

# Instantiate visualizer
tb = TensorBoardColab(graph_path='./log')

Wait for 8 seconds...
TensorBoard link:
http://2959fc26.ngrok.io


In [10]:




DEVICE='cuda:0'
LR=0.1
WEIGHT_DECAY = 0.000001 
MOMENTUM = 0.9
BATCH_SIZE = 256
TEST_BATCH_SIZE = 500
EPOCHS = 2

INPUT_DIM = 784
HIDDEN_DIM = 100
OUTPUT_DIM = 10

VIS_NAME = 'myFirstNN'


class MyFirstNetwork(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(MyFirstNetwork, self).__init__()
    self.input_to_hidden = nn.Linear(input_dim, hidden_dim)
    self.activation = nn.ReLU()
    self.hidden_to_output = nn.Linear(hidden_dim, output_dim)
    
    self.input_to_hidden.weight.data.normal_(0,0.1)
    self.input_to_hidden.bias.data.fill_(0.0)
    self.hidden_to_output.weight.data.normal_(0,0.1)
    self.hidden_to_output.bias.data.fill_(0.0)

  def forward(self, x):
    x = x.view(x.shape[0],-1)
    x = self.input_to_hidden(x)
    x = self.activation(x)
    x = self.hidden_to_output(x)
    return x
  

def get_data(): 
  transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

  full_training_data = torchvision.datasets.MNIST('.', train=True, download=True, transform=transform) 
  test_data = torchvision.datasets.MNIST('.', train=False, download=True, transform=transform)
  


  num_samples = len(full_training_data)
  training_samples = int(num_samples*0.5+1)
  validation_samples = num_samples - training_samples

  training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples, validation_samples])

  train_loader = torch.utils.data.DataLoader(training_data, batch_size = BATCH_SIZE, shuffle=True)
  val_loader = torch.utils.data.DataLoader(validation_data, batch_size = TEST_BATCH_SIZE, shuffle=False)
  test_loader = torch.utils.data.DataLoader(test_data, batch_size = TEST_BATCH_SIZE, shuffle=False)
  
  return train_loader, val_loader, test_loader


def get_optimizer(net, lr, wd, momentum):
  optimizer = optim.SGD(net.parameters(), lr=lr, weight_decay=wd, momentum=momentum)
  return optimizer


def get_cost_function():
  return nn.CrossEntropyLoss()



def test(net, data_loader, cost_function):
  net.eval()
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(data_loader):
      # Load data into GPU
      inputs = inputs.to(DEVICE)
      targets = targets.to(DEVICE)
        
      # Forward pass
      outputs = net(inputs)

      # Apply the loss
      loss = cost_function(outputs,targets)

      # Better print something, no?
      samples+=inputs.shape[0]
      cumulative_loss += loss.item()
      _, predicted = outputs.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100




def train(net,data_loader,optimizer,cost_function):
  net.train()
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  for batch_idx, (inputs, targets) in enumerate(data_loader):
    # Load data into GPU
    inputs = inputs.to(DEVICE)
    targets = targets.to(DEVICE)
      
    # Forward pass
    outputs = net(inputs)

    # Apply the loss
    loss = cost_function(outputs,targets)

    # Reset the optimizer
    optimizer.zero_grad()
      
    # Backward pass
    loss.backward()
      
    # Update parameters
    optimizer.step()

    # Better print something, no?
    samples+=inputs.shape[0]
    cumulative_loss += loss.item()
    _, predicted = outputs.max(1)
    cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100


def main():
  train_loader, val_loader, test_loader = get_data()
  net = MyFirstNetwork(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM).to(DEVICE)
  optimizer = get_optimizer(net, LR, WEIGHT_DECAY, MOMENTUM)
  cost_function = get_cost_function()

  print('Before training:')
  train_loss, train_accuracy = test(net, train_loader, cost_function)
  val_loss, val_accuracy = test(net, val_loader, cost_function)
  test_loss, test_accuracy = test(net, test_loader, cost_function)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')
  
  # Add values to plots
  tb.save_value('Train Loss',VIS_NAME, 0,train_loss)
  tb.save_value('Val Loss',VIS_NAME, 0,val_loss)
  tb.save_value('Train Accuracy',VIS_NAME, 0,train_accuracy)
  tb.save_value('Val Accuracy',VIS_NAME, 0,val_accuracy)
    
  # Update plots 
  tb.flush_line(VIS_NAME)
  tb.flush_line(VIS_NAME)
  
  for e in range(EPOCHS):
    train_loss, train_accuracy = train(net, train_loader, optimizer, cost_function)
    val_loss, val_accuracy = test(net, val_loader, cost_function)
    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')

    # Add values to plots
    tb.save_value('Train Loss',VIS_NAME, e+1,train_loss)
    tb.save_value('Val Loss',VIS_NAME, e+1,val_loss)
    tb.save_value('Train Accuracy',VIS_NAME, e+1,train_accuracy)
    tb.save_value('Val Accuracy',VIS_NAME, e+1,val_accuracy)
    
    # Update plots 
    tb.flush_line(VIS_NAME)
    tb.flush_line(VIS_NAME)
  
  
  print('After training:')
  train_loss, train_accuracy = test(net, train_loader, cost_function)
  val_loss, val_accuracy = test(net, val_loader, cost_function)
  test_loss, test_accuracy = test(net, test_loader, cost_function)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')
  
  # Close the visualizer
  #tb.close()

main()

Before training:
	 Training loss 0.00950, Training accuracy 14.85
	 Validation loss 0.00484, Validation accuracy 15.02
	 Test loss 0.00482, Test accuracy 16.16
-----------------------------------------------------
Epoch: 1
	 Training loss 0.00191, Training accuracy 85.12
	 Validation loss 0.00049, Validation accuracy 92.92
-----------------------------------------------------
Epoch: 2
	 Training loss 0.00080, Training accuracy 94.14
	 Validation loss 0.00037, Validation accuracy 94.44
-----------------------------------------------------
After training:
	 Training loss 0.00063, Training accuracy 95.44
	 Validation loss 0.00037, Validation accuracy 94.44
	 Test loss 0.00036, Test accuracy 94.54
-----------------------------------------------------
